<a href="https://colab.research.google.com/github/JakobUniver/algorithmics_3D_maze/blob/main/evolutionary_algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic algorithm
This algorithm goal is to find the best direction to thow a ball to hit the target.

In [1]:
# imports
import numpy as np
from copy import deepcopy
import time

In [5]:
np.array([1,2,3]).max()

3

In [2]:
# Helper functions

# https://stackoverflow.com/a/13849249/71522
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(vec1, vec2):
    """ Returns the angle in radians between unit-vectors 'vec1' and 'vec2' """
    return np.arccos(np.clip(np.dot(vec1, vec2), -1.0, 1.0))


def check_validity(room, index):
    for i in range(len(room.shape)):  # x, y, z
        if not (0 <= index[i] < room.shape[i]):            
            return 0
    if room[index[0], index[1], index[2]] == 3:
        return 0
    return 1

#https://math.stackexchange.com/questions/1905533/find-perpendicular-distance-from-point-to-line-in-3d
def get_distance(A, B, C):
    arg1 = A - B
    arg2 = C - B
    return np.linalg.norm(np.cross(arg1, arg2))/np.linalg.norm(arg2)

In [9]:
class Evolution:
    xz_range:tuple = 360
    yz_range:tuple = 360
    

    def __init__(self, population_size, room):
        self.population_size = population_size
        self.room = room
        self.population = self.init_population()
  
    def fitness(self, direction: np.array, path_output=False)->float:
        zx_rad, zy_rad = direction[0]*np.pi/180, direction[1]*np.pi/180
        x = np.sin(zx_rad)
        y = np.sin(zy_rad)
        z1, z2 = np.cos(zx_rad), np.cos(zy_rad)
        move_vector = unit_vector([np.around(x, 9), np.around(y, 9), np.around((z1+z2)/2, 9)])
        #print(f'Movement vector: {move_vector}')

        ball = np.where(self.room == 1)
        ball_start = np.array([ball[0][0], ball[1][0], ball[2][0]])
        #print(f'Ball starting at {ball_start}')
        end = np.where(self.room == 2)
        end = np.array([end[0][0], end[1][0], end[2][0]])

        directions = [1 if move_vector[0] >= 0 else -1,  # x
                      1 if move_vector[1] >= 0 else -1,  # y
                      1 if move_vector[2] >= 0 else -1]  # z
        curr_start = ball_start
        curr_pos = ball_start
        curr_len = 0
        path = [curr_start]
        max_len = 1000
        min_dist = np.inf

        while curr_len < max_len and min_dist != 0:
            #print(f'Iteration {curr_len+1}')

            candidates = [[curr_pos[0] + directions[0], curr_pos[1], curr_pos[2]],  # x
                          [curr_pos[0], curr_pos[1] + directions[1], curr_pos[2]],  # y
                          [curr_pos[0], curr_pos[1], curr_pos[2] + directions[2]]]  # z
            #print(f'Candidates - x:{candidates[0]}, y:{candidates[1]}, z:{candidates[2]}')

            cand_vectors = [unit_vector(np.subtract(candidate, curr_start)) for candidate in candidates]

            angles = [angle_between(cand_vec, move_vector) for cand_vec in cand_vectors]
            #print(f'Angles - x:{angles[0]}, y:{angles[1]}, z:{angles[2]}')

            min_angle_arg = np.argmin(angles)
            valid = check_validity(self.room, candidates[min_angle_arg])

            if valid:  # if movement is valid 
                curr_pos = candidates[min_angle_arg]
                #print(f'Moving to {curr_pos}')
                curr_len += 1

            else:  # if movement is not valid and ball must bounce
                distance = get_distance(end, np.array(curr_start), np.array(curr_pos))
                if distance < min_dist:
                    min_dist = distance

                directions[min_angle_arg] *= -1
                move_vector[min_angle_arg] *= -1
                curr_start = curr_pos
                path.append(curr_start)

        #print(f'Ending movement in {curr_pos}')
        if path_output:
            path.append(curr_pos)
            return min_dist, path
        else:
            return min_dist

    def init_population(self)->list:
        population = np.random.random((self.population_size,2))
        # Next we do because we do not want that the range endpoints are included
        population[:,0] = population[:,0]*(self.xz_range/2)
        population[:,1] = population[:,1]*(self.yz_range/2)
        population[:,0] = population[:,0]*np.random.choice([-1,1],self.population_size)
        population[:,1] = population[:,1]*np.random.choice([-1,1],self.population_size)
        population_valuation = np.array([self.fitness(chromosome) for chromosome in population])
        return population_valuation, population

    def mutation(self,chromosome:np.array, nr_genes:int=6)->tuple:
        # https://en.wikipedia.org/wiki/Mutation_(genetic_algorithm)
        xz_dif = np.random.normal(0,self.xz_range/nr_genes)
        yz_dif = np.random.normal(0,self.yz_range/nr_genes)

        mutant = np.array([chromosome[0]+xz_dif,chromosome[1]+yz_dif])

        if abs(mutant[0])>self.xz_range/2:
            mutant[0] = (self.xz_range - abs(mutant[0])) *(1 if mutant[0]<0 else -1)
        if abs(mutant[1])>self.yz_range/2:
            mutant[1] = (self.xz_range - abs(mutant[1])) *(1 if mutant[1]<0 else -1)

        return self.fitness(mutant),mutant

    def crossover(self,parents:np.array)->list:
        parent1,parent2 = parents

        child1_val,child1 = self.mutation([parent1[0],parent2[1]])
        child2_val,child2 = self.mutation([parent2[0],parent1[1]])

        return [child1_val,child2_val],[child1,child2]

    def generate_pairs(self,parents:tuple,size:int)-> list:
        parent_valuations,parent_individuals = parents
        valuations = parent_valuations.max() + 1 - parent_valuations # Make smaller values bigger and bigger values smaller
        parents_probabilites = valuations / valuations.sum()
        chosen_parents_i = np.random.choice(range(len(parents_probabilites)),size*2,p=parents_probabilites)# ,replace=False
        chosen_parents = parent_individuals[chosen_parents_i]
        pairs = np.array(list(zip(chosen_parents[::2],chosen_parents[1::2])))
        return pairs
  
    def generate_offspring(self)->tuple:
        parents = self.generate_pairs(self.population,self.population_size//4)
        children_val= []
        children= []
        for pair in parents:
            children_val_i, children_i=self.crossover(pair)
            children_val+=children_val_i
            children+=children_i

        return children_val,children_i

    def choose_new_generation(self, population:tuple)->tuple:
        valuations = population[0].max() + 1 - population[0]# Make smaller values bigger and bigger values smaller
        chromo_probabilites = valuations / valuations.sum()
        chosen_chromos_i = np.random.choice(range(len(chromo_probabilites)),self.population_size,p=chromo_probabilites,replace=False)
        new_population = (population[0][chosen_chromos_i],population[1][chosen_chromos_i])
        return new_population

    
    def step(self)->list:
        children = []

        # Generate new offspring
        children = self.generate_offspring()
        new_population_val = np.concatenate([self.population[0],children[0]])
        new_population_ins = np.concatenate([self.population[1],children[1]])
        new_population = (new_population_val,new_population_ins)

        # Let the natural selection to do its job
        self.population = self.choose_new_generation(new_population)

        return self.population

    def step_until(self,max_iters:int=1000, max_non_increasing_iters:int=100, max_time_s:int=10):
        iters = 0
        non_increasing_iters = 0
        best_val = float('inf')
        progress_vals = []
        progress_ins = []
        start_time = time.perf_counter()

        while True:
            # Check for reasons to stop evolution
            if iters == max_iters:
                print('Stopped because maximum iterations achieved.')
                break
            if non_increasing_iters == max_non_increasing_iters:
                print('Stopped because maximum non increasing iterations achieved.')
                break
            if time.perf_counter() - start_time >= max_time_s:
                print('Stopped because maximum time limit achieved.')
                break

            # Produce the next generation
            generation_i_vals,generation_i_ins = self.step()

            # Find the best individual
            best_i_idx = generation_i_vals.argmin()
            best_i_val = generation_i_vals[best_i_idx]
            best_i_in = generation_i_ins[best_i_idx]
            progress_vals.append(best_i_val)
            progress_ins.append(best_i_in)

            # Increase counters
            iters +=1
            non_increasing_iters +=1

            # Check if there is an improvement
            if best_i_val < best_val:
                best_val=best_i_val
                non_increasing_iters = 0

        return np.array(progress_vals),np.array(progress_ins)
      
        
matrix = np.zeros((11, 11, 11), int)

# 0 - Empty cube
# 1 - Start cube
# 2 - End cube
# 3 - Obstacle

matrix[0][5][5] = 1
matrix[10][5][5] = 2
matrix[5][4:6] = 3

obj = Evolution(5, room=matrix)
print(f'Population size: {obj.population_size}')
print(f'Population:\n{obj.population}')
print()
print(obj.mutation([0,0]))
print()
print(obj.crossover([[-45,45],[0,-30]]))
print()
print(obj.generate_pairs(
    (np.array([10,    25,     23,      15,      100]), 
     np.array([[-4,5],[12,23],[-15,25],[75,-55],[-36,23]])
    ), 3)
     )

print()
print(obj.generate_offspring())
print()
obj.step()
print()
obj.step_until()

<ipython-input-2-0c1cdb657041>:24: RuntimeWarning: invalid value encountered in double_scalars
  return np.linalg.norm(np.cross(arg1, arg2))/np.linalg.norm(arg2)


Population size: 5
Population:
(array([0.66666667, 0.98802352, 0.6       , 0.83205029, 2.39869367]), array([[ 142.86097055,   -9.2187276 ],
       [ 130.79262731,  -61.59545435],
       [  10.65671227,  168.46264252],
       [ -55.44203954,    3.51987987],
       [ 167.11110661, -134.26059396]]))

(1.185226520443204, array([ 12.64645431, -66.21716714]))

([0.0, 1.2068644245027835], [array([-58.04849064, -67.89420033]), array([-24.3415641 , 133.88214039])])

parent_valuations[ 10  25  23  15 100]
valuations[91 76 78 86  1]
[[[ -4   5]
  [-15  25]]

 [[ -4   5]
  [ 75 -55]]

 [[ -4   5]
  [ 75 -55]]]

parent_valuations[0.66666667 0.98802352 0.6        0.83205029 2.39869367]
valuations[2.73202701 2.41067015 2.79869367 2.56664338 1.        ]
([4.510968544481587, 1.224744871391589], [array([140.20575225,  18.95709412]), array([-107.00994694,  -98.26121761])])

parent_valuations[0.66666667 0.98802352 0.6        0.83205029 2.39869367]
valuations[2.73202701 2.41067015 2.79869367 2.56664338 1. 

(array([0. , 0.6, 0.6, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. ]), array([[ 174.5459256 ,   -7.53964459],
        [  10.65671227,  168.46264252],
        [  10.65671227,  168.46264252],
        [ 142.75678886,  -93.99393211],
        [ 142.75678886,  -93.99393211],
        [ 142.75678886,  -93.99393211],
        [ 145.585782  ,  101.20325612],
        [ 145.585782  ,  101.20325612],
        [-165.06574361,   63.67569767],
        [ 138.50875568,   78.12941063],
        [-165.06574361,   63.67569767],
        [-165.06574361,   63.67569767],
        [-165.06574361,   63.67569767],
        [-171.98690882,   11.77694761],
        [-171.98690882,   11.77694761],
        [-171.98690882,   11.77694761],
        [-171.98690882,   11.77694761],
        [-171.98690882,   11.77694761],
        [-171.98690882,   11.776